In [ ]:
!pip install pulp

!sudo apt-get install coinor-cbc glpk-utils coinor-clp

     |████████████████████████████████| 40.6MB 107kB/s 
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  coinor-libcbc3 coinor-libcgl1 coinor-libclp1 coinor-libcoinutils3v5
  coinor-libosi1v5 libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev default-libmysqlclient-dev
The following NEW packages will be installed:
  coinor-cbc coinor-clp coinor-libcbc3 coinor-libcgl1 coinor-libclp1
  coinor-libcoinutils3v5 coinor-libosi1v5 glpk-utils libamd2 libcolamd2
  libglpk40 libsuitesparseconfig5
0 upgraded, 12 newly installed, 0 to remove and 14 not upgraded.
Need to get 3,439 kB of archives.
After this operation, 9,832 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 libsuitesparseconfig5 amd64 1:5.1.2-2 [9,044 B]
Get:2 http://archive.ubuntu.com/ubuntu bionic/main amd64 libamd2 amd64 1:5.1.2-2 [19.5 kB]
Get:3 

In [1]:
import pulp
print(pulp.__version__)

2.3.1


Minimalne straty w mostku

| Rezystor     | Napięcie[v] |prąd[mA]|
|------------|-------------------||
| R1    | 6              |4|
|R2|10|2|
| R3     | 4              | 2|
| R4       | 7                |2|
| R5 | 3                 |4|

In [ ]:
from pulp import *

U = [6,10,4,7,3]

prob = LpProblem("MiinimalnaMoc",LpMinimize)

i1=LpVariable("i1",0,None,LpContinuous)
i2=LpVariable("i2",0,None,LpContinuous)
i3=LpVariable("i3",0,None,LpContinuous)
i4=LpVariable("i4",0,None,LpContinuous)
i5=LpVariable("i5",0,None,LpContinuous)

prob += 6*i1 + 10*i2 + 4*i3 + 7*i4 + 3*i5


prob += i1+i2 == i4+i5
prob += i1 == i3+i4
prob += i2 == i5-i3


prob += i1 >= 3
prob += i1 <= 5
prob += i2 >= 1
prob += i2 <= 3
prob += i3 >= 1
prob += i3 <= 3
prob += i4 >= 1
prob += i4 <= 3
prob += i5 >= 3
prob += i5 <= 5

prob.writeLP("MiinimalnaMoc.lp")
prob.solve()

varsdict = {}

j = 0
for v in prob.variables():
    varsdict[v.name,"R"+str(j+1)] = str(v.varValue), U[j]/v.varValue
    j += 1


print("Status:", LpStatus[prob.status])
print("minalna moc: {} W".format(value(prob.objective)))
print("[A] : [Ohm]: {}".format(varsdict))

Status: Optimal
minalna moc: 52.0 W
[A] : [Ohm]: {('i1', 'R1'): ('3.0', 2.0), ('i2', 'R2'): ('1.0', 10.0), ('i3', 'R3'): ('2.0', 2.0), ('i4', 'R4'): ('1.0', 7.0), ('i5', 'R5'): ('3.0', 1.0)}


Maksymalny prad, który może popłynąć przez układ rezystorów

|     | R |I |     
|------------|-------------------|----------
| R1    |   8      | 2 |
|   R2       |    6               |   3  
|   R3      |    4               |   4  
|   R4      |    10|2
|R5|8|2


In [ ]:
from pulp import *

#problem statement
prob = LpProblem("maksDwojnika",LpMaximize)

I1 = LpVariable("I1",0,None,LpContinuous)
I2 = LpVariable("I2",0,None,LpContinuous)
I3 = LpVariable("I3",0,None,LpContinuous)
I4 = LpVariable("I4",0,None,LpContinuous)
I5 = LpVariable("I5",0,None,LpContinuous)

prob += I3, "SumaPradow" 

prob += I1 + I2 == I3, "suma1"
prob += I5 + I4 == I3, "suma2"

prob += I1*8 == I2*6, "Ohm1"
prob += I5*8 == I4*10, "Ohm2"

prob += I1 <= 2, "Ogr_I1"
prob += I2 <= 3, "Ogr_I2"
prob += I3 <= 4, "Ogr_I3"
prob += I4 <= 2, "Ogr_I4"
prob += I5 <= 2, "Ogr_I5"


prob.solve()
print ("Status:", LpStatus[prob.status])
print()
print("Maksymalny prad dwojnika",prob.objective.value(),"A")
print()
print("Wartości prądów gdzie maksymalizowany jest prąd dwójnika")
for v in prob.variables():
    print (v.name, "=", v.varValue)


Status: Optimal

Maksymalny prad dwojnika 3.6 A

Wartości prądów gdzie maksymalizowany jest prąd dwójnika
I1 = 1.5428571
I2 = 2.0571429
I3 = 3.6
I4 = 1.6
I5 = 2.0
